In [ ]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

Load Data

In [ ]:
file_name = 'file_name'
raw_df = pd.read_csv(rf'C:\Users\chaiyaphop.jam\Documents\Works\Markov_Chains\Data_Set\{file_name}.csv')
raw_df.head()

Select Features

In [ ]:
df = raw_df[['PACKAGE_01','PACKAGE_02','PACKAGE_03','PACKAGE_04','PACKAGE_05','PACKAGE_06']]
df.describe().round(2)

Remove Outlier

In [ ]:
# outlier_p1 = df.PACKAGE_01 > df.PACKAGE_01.quantile(1)
# outlier_p2 = df.PACKAGE_02 > df.PACKAGE_02.quantile(1)
# outlier_p3 = df.PACKAGE_03 > df.PACKAGE_03.quantile(1)
# outlier_p4 = df.PACKAGE_04 > df.PACKAGE_04.quantile(1)
# outlier_p5 = df.PACKAGE_05 > df.PACKAGE_05.quantile(1)
# outlier_p6 = df.PACKAGE_06 > df.PACKAGE_06.quantile(1)
# outlier_p1 = df.PACKAGE_01 > df.PACKAGE_01.quantile(1)
# outlier_p2 = df.PACKAGE_02 > df.PACKAGE_02.quantile(1)
# outlier_p3 = df.PACKAGE_03 > df.PACKAGE_03.quantile(1)
# outlier_p4 = df.PACKAGE_04 > df.PACKAGE_04.quantile(1)
# outlier_p5 = df.PACKAGE_05 > df.PACKAGE_05.quantile(1)
# outlier_p6 = df.PACKAGE_06 > df.PACKAGE_06.quantile(1)

# flag_outlier = outlier_p1 | outlier_p2 | outlier_p3 | outlier_p4 | outlier_p5 | outlier_p6

# df_clean = df.loc[~flag_outlier]
# print(df_clean.describe().round(2))
# print(df_clean.corr())

Scatter Plot

In [ ]:
# sns.pairplot(df_clean, diag_kind = 'kde')

Sample Data

In [ ]:
df_array = np.array(df)
df_array[:5]

Possible State and Encode Possible State

In [ ]:
all_state = []
for row in df_array:
    for col in row:
        all_state.append(col)

possible_state = np.sort(np.unique(all_state)).tolist()
print(f'Possible State is {possible_state}')

encoded_possible_state = []
for state in possible_state:
    for i in range(len(possible_state)):
        if possible_state.index(state) == i:
            encoded_possible_state.append(i)
print(f'Encoded Possible State is {encoded_possible_state}')

n_possible_state = 0
for i in encoded_possible_state:
    n_possible_state += 1
print(f'Number of Possible State is {n_possible_state}')

Encode Features

In [ ]:
for i in possible_state:
    np.place(df_array, df_array == i, [possible_state.index(i)])
df_train = df_array
df_train[:5]

Markov Chain Model

In [ ]:
# Estimate Transition Prob. Matrix function
def transition_prob_matrix(df_train):
    est_trans_prob = np.zeros((n_possible_state,n_possible_state))

    for row in range(est_trans_prob.shape[0]):
        for col in range(est_trans_prob.shape[1]):
            target_transition = [row,col]

            for each_step in range(1,df_train.shape[1]):
                this_transition = np.array(df_train)[:,[each_step-1,each_step]].tolist()

                for each_seq in this_transition:
                    if each_seq == target_transition:
                        est_trans_prob[row,col] += 1

            est_trans_prob[row,col] /= len(np.argwhere(np.array(df_train)[:,:-1] == row))
    
    return est_trans_prob.round(2)

# Predictive function
def predictor(est_trans_prob, train_seq):
    next_state_list = []
    for each_seq in range(pd.DataFrame(df_train).count()[1]):
        current_state = train_seq[each_seq][-1]

        next_state = np.argmax(est_trans_prob[current_state,:])
        next_state_list.append(next_state)

    return next_state_list

In [ ]:
# Train Test Split
train_seq = np.array(df_train)[:,:-1]
test_seq = np.array(df_train)[:,-1]

# Transition Prob. Matrix Estimation
est_trans_prob = transition_prob_matrix(df_train)
df_est_trans_prob = pd.DataFrame(est_trans_prob)
df_est_trans_prob.index = possible_state
df_est_trans_prob.columns = possible_state
export_trans_prob = df_est_trans_prob.to_csv(rf'C:\Users\chaiyaphop.jam\Documents\Works\Markov_Chains\Result\Trans_Prob_{file_name}.csv')
print('Transition Prob. Matrix')
print(df_est_trans_prob)

# Prediction
pred_seq = predictor(est_trans_prob, train_seq)

# Accuracy
print('%Accuracy :',round((pred_seq == test_seq).sum()/len(test_seq)*100),'%')

Prediction

In [ ]:
# Actual Data Set
current_state = np.array(df_array)
last_state = np.array(df_array)[:,-1]

# Prediction
future_state = predictor(est_trans_prob, current_state)

# %Change
print('%Change from Last State :',round((1-((future_state == last_state).sum()/len(last_state)))*100),'%')

Export

In [ ]:
# Decode Future State
decode_future_state = []

for i in future_state:
    if i == possible_state.index(possible_state[i]):
        decode_future_state.append(possible_state[i])
future_state_df = pd.DataFrame(decode_future_state)
future_state_df.columns = ['PRED_PACKAGE']

result = pd.concat([raw_df,future_state_df], axis=1)

# Export
export_result = result.to_csv(rf'C:\Users\chaiyaphop.jam\Documents\Works\Markov_Chains\Result\Result_Markov_{file_name}.csv')
result